## 1. 聚合函数
- 分组后得到的GroupBy对象可以使用聚合函数进行数据聚合，以下是一些常用的聚合函数：
![title](img/数据聚合.png)
- GroupBy对象还可以调用被分组对象里自带的任何函数（非聚合函数也可以）
- 自定义的聚合函数，只需使用`aggregate()`或`agg()`方法传入自定义函数，分组结果的各个切片调用自定义函数并返回运算结果
- 上表中的聚合函数，其实就是使用`agg()`方法调用聚合函数，被调用的聚合函数名称以字符串形式传入`agg()`，如`agg('mean')`
- 返回结果禁用索引：聚合运算后得到的结果默认都是以分组键作为索引，可在分组时使用`as_index=False`禁止将分组键作为索引而只作为普通的列，相当于对结果`reset_index()`

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({
    'key1': list('ABBCBCAA'),
    'key2': list('YZXYZXYZ'),
    'data1': np.random.randint(100, size=8),
    'data2': np.random.randint(10, size=8)
})
df

,key1,key2,data1,data2
0,A,Y,17,0
1,B,Z,62,3
2,B,X,34,0
3,C,Y,59,0
4,B,Z,39,5
5,C,X,0,5
6,A,Y,46,2
7,A,Z,11,2


In [3]:
# 自定义函数进行聚合操作
def test(arr):
    if len(arr) > 1:
        return max(arr) - min(arr)
    else:
        return arr


df.groupby(['key1', 'key2']).agg(test)

data1  data2
key1 key2              
A    Y        29      2
     Z        11      2
B    X        34      0
     Z        23      2
C    X         0      5
     Y        59      0

In [4]:
# 以下语句相当于df.groupby(['key1','key2']).mean()
df.groupby(['key1', 'key2']).agg('mean')

data1  data2
key1 key2              
A    Y      31.5    1.0
     Z      11.0    2.0
B    X      34.0    0.0
     Z      50.5    4.0
C    X       0.0    5.0
     Y      59.0    0.0

In [5]:
# 分组时可以禁止将分组键作为索引
df.groupby(['key1', 'key2'], as_index=False).agg('mean')

,key1,key2,data1,data2
0,A,Y,31.5,1.0
1,A,Z,11.0,2.0
2,B,X,34.0,0.0
3,B,Z,50.5,4.0
4,C,X,0.0,5.0
5,C,Y,59.0,0.0


## 2.  应用多个聚合函数
如果需要对数据中不同的列分别应用不同的聚合函数，可以使用`agg()`来实现
- **对所有列应用多个聚合函数：**  
将多个函数的函数名以数组形式传入`agg()`，返回结果中的列将会以函数名来命名
- 若需要自定义返回的列名，可以传入一组元组列表，元组元素分别为自定义名称和函数名
- **对每个列应用不同聚合函数：**  
以字典形式传入列名和函数名，其中列名作为key，函数名作为value，多个函数的话由函数名组成列表，若要返回自定义列名则使用元组，`{'列名':[('自定义列名','函数名'),'函数名']}`
- 只有在列上应用了多个函数时，才会返回层次化数据

In [6]:
# 同时应用多种聚合函数，注意已有的聚合函数名以字符串形式传入，列名默认为函数名
df.groupby(['key1', 'key2']).agg(['mean', 'count', test])

data1            data2           
           mean count test  mean count test
key1 key2                                  
A    Y     31.5     2   29     1     2    2
     Z     11.0     1   11     2     1    2
B    X     34.0     1   34     0     1    0
     Z     50.5     2   23     4     2    2
C    X      0.0     1    0     5     1    5
     Y     59.0     1   59     0     1    0

In [7]:
# 应用多种聚合函数，并自定义列名
df.groupby(['key1', 'key2']).agg([('平均数', 'mean'), ('计数', 'count')])

data1    data2   
            平均数 计数   平均数 计数
key1 key2                  
A    Y     31.5  2     1  2
     Z     11.0  1     2  1
B    X     34.0  1     0  1
     Z     50.5  2     4  2
C    X      0.0  1     5  1
     Y     59.0  1     0  1

In [8]:
# 对每个列应用不同聚合函数
df.groupby(['key1', 'key2']).agg({
    'data1': [('最小', 'min'), ('计数', 'count')],
    'data2': ['mean', test]
})

data1    data2     
             最小 计数  mean test
key1 key2                    
A    Y       17  2     1    2
     Z       11  1     2    2
B    X       34  1     0    0
     Z       39  2     4    2
C    X        0  1     5    5
     Y       59  1     0    0

## 3. apply()方法
`apply(func)` 对已经被拆分成多个片段的GroupBy对象，将函数func应用到每个片段上，最后将得到的结果再组合到一起，如果函数func还需要其他参数，将这些参数放在函数名后一起传入。  
默认情况下分组键会和原索引共同构成运算后结果的索引，可以使用`group_keys=False`禁止分组键作为索引
**`apply()`和`agg()`区别：**  
- `apply()` 是将从表格中拆分出来的子表（DataFrame或者Series）应用到传入的函数上
- `agg()` 是将表格的一列数据应用到传入的函数上

In [9]:
df

,key1,key2,data1,data2
0,A,Y,17,0
1,B,Z,62,3
2,B,X,34,0
3,C,Y,59,0
4,B,Z,39,5
5,C,X,0,5
6,A,Y,46,2
7,A,Z,11,2


In [10]:
# 建立一个函数，将传入的数据按照指定列进行排序
def func_sort(df, columns='data1'):
    return df.sort_values(columns)


# 使用apply传入函数名，并在函数名后传入参数
df.groupby(['key1', 'key2']).apply(func_sort, columns=['data1', 'data2'])

key1 key2  data1  data2
key1 key2                          
A    Y    0    A    Y     17      0
          6    A    Y     46      2
     Z    7    A    Z     11      2
B    X    2    B    X     34      0
     Z    4    B    Z     39      5
          1    B    Z     62      3
C    X    5    C    X      0      5
     Y    3    C    Y     59      0

In [11]:
# 上述方法可以看作是将GroupBy对象中每个元素的数据部分应用func_sort()函数，最后再将结果组合起来
for x, y in df.groupby(['key1', 'key2']):
    print(x)
    print(func_sort(y, columns=['data1', 'data2']))  # 对每个数据部分应用函数

('A', 'Y')
  key1 key2  data1  data2
0    A    Y     17      0
6    A    Y     46      2
('A', 'Z')
  key1 key2  data1  data2
7    A    Z     11      2
('B', 'X')
  key1 key2  data1  data2
2    B    X     34      0
('B', 'Z')
  key1 key2  data1  data2
4    B    Z     39      5
1    B    Z     62      3
('C', 'X')
  key1 key2  data1  data2
5    C    X      0      5
('C', 'Y')
  key1 key2  data1  data2
3    C    Y     59      0


In [12]:
# 禁止分组键作为索引
df.groupby(['key1', 'key2'], group_keys=False).apply(
    func_sort, columns=['data1', 'data2'])

,key1,key2,data1,data2
0,A,Y,17,0
6,A,Y,46,2
7,A,Z,11,2
2,B,X,34,0
4,B,Z,39,5
1,B,Z,62,3
5,C,X,0,5
3,C,Y,59,0


In [13]:
# apply对于Series同样适用
f = lambda x: x.sort_values()
df['data1'].groupby(list('AABBABAA')).apply(f)

A  7    11
   0    17
   4    39
   6    46
   1    62
B  5     0
   2    34
   3    59
Name: data1, dtype: int32

## 3. 分位数和桶分析
pandas有如`cut()`和`qcut()`这种函数可以将数据根据指定面元或样本分位数拆分成多块，拆分后的数据可以直接传给`groupby()`，实现对数据的桶分析或分位数分析。  
- 桶拆分：根据面元建立相应数量和长度的桶，或根据拆分数量建立同数量的长度相等的桶，然后将数据拆分后装入对应桶中，根据数值划分，如`cut()`
- 分位数拆分：根据样本分位数得到大小相等的桶，根据元素数量划分，如`qcut()`  

拆分后的对象代入`groupby()`中，根据拆分出来的块对数据进行分组

In [14]:
frame = pd.DataFrame({
    'data1': np.random.randint(100, size=100),
    'data2': np.random.randint(100, size=100)
})

In [15]:
# 使用cut将数据拆分成4个等长的块
cut_frame = pd.cut(frame['data1'], 4)
cut_frame.head(10)

0    (24.75, 49.5]
1    (24.75, 49.5]
2    (24.75, 49.5]
3    (74.25, 99.0]
4    (49.5, 74.25]
5    (74.25, 99.0]
6    (49.5, 74.25]
7    (74.25, 99.0]
8    (74.25, 99.0]
9    (24.75, 49.5]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-0.099, 24.75] < (24.75, 49.5] < (49.5, 74.25] < (74.25, 99.0]]

In [16]:
# 拆分结果可以直接代入groupby中，根据拆分结果进行分组
frame.groupby(cut_frame).agg({'data1':['mean','count'],'data2':'sum'})

data1       data2
                      mean count   sum
data1                                 
(-0.099, 24.75]  12.208333    24  1124
(24.75, 49.5]    35.366667    30  1606
(49.5, 74.25]    59.111111    27  1380
(74.25, 99.0]    87.684211    19   860

In [17]:
# 使用分位数划分数量相同的4段
qcut_frame=pd.qcut(frame['data2'],4)
qcut_frame.name='分位数' # 修改name属性，作为分组之后的索引名
frame.groupby(qcut_frame).agg({'data2':['mean','count'],'data1':'sum'})

data2       data1
                    mean count   sum
分位数                                 
(0.999, 26.5]  11.840000    25  1087
(26.5, 46.0]   35.720000    25  1222
(46.0, 80.0]   63.307692    26  1269
(80.0, 98.0]   88.958333    24  1038

In [21]:
frame.groupby(qcut_frame).agg(['max','min','count'])

data1           data2          
                max min count   max min count
分位数                                          
(0.999, 26.5]    95   3    25    25   1    25
(26.5, 46.0]     99   8    25    45  27    25
(46.0, 80.0]     95   0    26    80  47    26
(80.0, 98.0]     90   4    24    98  81    24